In [1]:
import pandas as pd
import numpy as np
import os
import gzip
import time
import seaborn as sns
import matplotlib.pyplot as plt
import ldspec

%load_ext autoreload
%autoreload 2

In [2]:
DATA_PATH = '/n/groups/price/martin/LDSPEC_data/UKBimp_337K_MAF001_chimp/pannot_fct'
temp_dic = {
#     # Unstrat
#     'baseline_0_100' : DATA_PATH + '/baseline_0_100/@a.proxy_0_100.ld_full.maf_@m_block.c@.pannot_mat.npz',
#     'baseline_0_1000' : DATA_PATH + '/baseline_0_1000/@a.proxy_0_1000.ld_full.maf_@m_block.c@.pannot_mat.npz',
#     # pos-ld
#     'baseline_0_100_ld_pos' : DATA_PATH + '/baseline_0_100_ld_pos/@a.proxy_0_100.ld_p0_p100.maf_@m_block.c@.pannot_mat.npz',
#     'baseline_0_1000_ld_pos' : DATA_PATH + '/baseline_0_1000_ld_pos/@a.proxy_0_1000.ld_p0_p100.maf_@m_block.c@.pannot_mat.npz',
#     # neg-ld
#     'baseline_0_100_ld_neg' : DATA_PATH + '/baseline_0_100_ld_neg/@a.proxy_0_100.ld_n100_p0.maf_@m_block.c@.pannot_mat.npz',
#     'baseline_0_1000_ld_neg' : DATA_PATH + '/baseline_0_1000_ld_neg/@a.proxy_0_1000.ld_n100_p0.maf_@m_block.c@.pannot_mat.npz',
    # pos-high-ld
    'baseline_0_100_ld_high_pos' : DATA_PATH + '/baseline_0_100_ld_high_pos/@a.proxy_0_100.ld_p50_p100.maf_@m_block.c@.pannot_mat.npz',
    'baseline_0_1000_ld_high_pos' : DATA_PATH + '/baseline_0_1000_ld_high_pos/@a.proxy_0_1000.ld_p50_p100.maf_@m_block.c@.pannot_mat.npz',
}

DIC_PANNOT = {}
for key in temp_dic:
    for mbin in ['common', 'lf']:
        DIC_PANNOT['%s_%s'%(key,mbin)] = temp_dic[key].replace('@m',mbin)
        
for val in DIC_PANNOT.values():
    print(val)

/n/groups/price/martin/LDSPEC_data/UKBimp_337K_MAF001_chimp/pannot_fct/baseline_0_100_ld_high_pos/@a.proxy_0_100.ld_p50_p100.maf_common_block.c@.pannot_mat.npz
/n/groups/price/martin/LDSPEC_data/UKBimp_337K_MAF001_chimp/pannot_fct/baseline_0_100_ld_high_pos/@a.proxy_0_100.ld_p50_p100.maf_lf_block.c@.pannot_mat.npz
/n/groups/price/martin/LDSPEC_data/UKBimp_337K_MAF001_chimp/pannot_fct/baseline_0_1000_ld_high_pos/@a.proxy_0_1000.ld_p50_p100.maf_common_block.c@.pannot_mat.npz
/n/groups/price/martin/LDSPEC_data/UKBimp_337K_MAF001_chimp/pannot_fct/baseline_0_1000_ld_high_pos/@a.proxy_0_1000.ld_p50_p100.maf_lf_block.c@.pannot_mat.npz


In [3]:
# List of AN,pAN pairs where all 22 chrs are complete
# AN_list
temp_df = ldspec.util.read_annot(
    '/n/groups/price/martin/LDSPEC_data/UKBimp_337K_MAF001/baseline_annot/baseline_165annots_chr22.annot.gz'
)
AN_list = []
for col in temp_df:
    if col.startswith('AN:') is False:
        continue
    if 'flanking' in col:
        continue
    if 'mbin' in col:
        continue
    if 'all' in col:
        continue
    if set(temp_df[col]) != set([0,1]):
        continue
    if col.endswith('_common') is False:
        continue

    AN = col.replace('_common', '').replace('AN:', '')
    AN_list.append(AN)
print('AN_list %d' % len(AN_list))

# (AN,pAN) pairs with all CHRs
AN_pAN_list = []
for AN,pAN in [(x,y) for x in AN_list for y in DIC_PANNOT]:
    flag_keep = True
    for CHR in range(1,23):
#     for CHR in [22]:
        fpath = DIC_PANNOT[pAN].replace('@a', AN).replace('@', '%d' % CHR)
        if os.path.exists(fpath) is False:
            print('Skip %s %s' % (AN, pAN))
            flag_keep = False
            break
    if flag_keep:
        AN_pAN_list.append((AN, pAN))
print('Remove incomplete, AN_pAN_list %d' % len(AN_pAN_list))

AN_list 38
Skip CADD_g20 baseline_0_100_ld_high_pos_common
Skip CADD_g20 baseline_0_100_ld_high_pos_lf
Skip CADD_g20 baseline_0_1000_ld_high_pos_common
Skip CADD_g20 baseline_0_1000_ld_high_pos_lf
Skip UTR_5_UCSC baseline_0_100_ld_high_pos_lf
Skip snpeff_nonsynonymous_variant baseline_0_100_ld_high_pos_common
Skip snpeff_nonsynonymous_variant baseline_0_100_ld_high_pos_lf
Skip snpeff_nonsynonymous_variant baseline_0_1000_ld_high_pos_lf
Skip snpeff_synonymous_variant baseline_0_100_ld_high_pos_common
Skip snpeff_synonymous_variant baseline_0_100_ld_high_pos_lf
Skip snpeff_synonymous_variant baseline_0_1000_ld_high_pos_common
Skip snpeff_synonymous_variant baseline_0_1000_ld_high_pos_lf
Remove incomplete, AN_pAN_list 140


In [4]:
# # Computed n pair: C-C and L-L, for unstratified version 
# df_info = pd.DataFrame(index=AN_list, columns=[x for x in DIC_PANNOT if 'ld' not in x], data=0)
# for AN,pAN in AN_pAN_list:
#     if pAN not in df_info:
#         continue
#     for CHR in range(1,23):
#         fpath = DIC_PANNOT[pAN].replace('@a', AN).replace('@', '%s' % CHR)
#         if os.path.exists(fpath):
#             mat_G = ldspec.util.read_pannot_mat(fpath)
#             df_info.loc[AN, pAN] += mat_G.sum()

In [5]:
# # Write pannot list
# for pAN_pf in ['baseline_0_100', 'baseline_0_1000']:
#     AN_list_include = []
#     for AN in sorted(AN_list):
#         if df_info.loc[AN, ['%s_common'%pAN_pf, '%s_lf'%pAN_pf]].sum() > 1e6:  # 
#             AN_list_include.append(AN)
#     print(pAN_pf, len(AN_list_include))
#     print(','.join(AN_list_include))
    
#     # unstrat
#     with open('./pannot_list/pannot_list_%s.txt' % pAN_pf, 'w') as f: # write file
#         for AN in AN_list_include:
#             for mbin in ['common', 'lf']:
#                 fpath = '%s' % DIC_PANNOT['%s_%s' % (pAN_pf, mbin)].replace('@a', AN)
#                 f.write('%s\n' % fpath)
#     # ld_pos & ld_neg
#     with open('./pannot_list/pannot_list_%s_ld.txt' % pAN_pf, 'w') as f: # write file
#         for AN in AN_list_include:
#             for mbin in ['common', 'lf']:
#                 fpath = '%s' % DIC_PANNOT['%s_ld_pos_%s' % (pAN_pf, mbin)].replace('@a', AN)
#                 f.write('%s\n' % fpath)
#         for AN in AN_list_include:
#             for mbin in ['common', 'lf']:
#                 fpath = '%s' % DIC_PANNOT['%s_ld_neg_%s' % (pAN_pf, mbin)].replace('@a', AN)
#                 f.write('%s\n' % fpath)                
#     print('')

### Check correctness 

In [10]:
# # Compare with previous results 
# CHR = 22
# # for pannot in ['exon', 'exonic_gene', 'cS2G_promoter', 'protein_domain', 'gene']:
# for AN,pAN in [['Conserved_LindbladToh', 'baseline_0_1000']]:
#     print(pAN)
#     mat_G = ldspec.util.read_pannot_mat(DIC_PANNOT['%s_common'%pAN].replace("@a", AN).replace("@", "%d"%CHR))
#     mat_G_pos = ldspec.util.read_pannot_mat(DIC_PANNOT['%s_ld_pos_common'%pAN].replace("@a", AN).replace("@", "%d"%CHR))
#     mat_G_neg = ldspec.util.read_pannot_mat(DIC_PANNOT['%s_ld_neg_common'%pAN].replace("@a", AN).replace("@", "%d"%CHR))
#     mat_G_ref = ldspec.util.read_pannot_mat(
#         '/n/groups/price/martin/LDSPEC_data/UKBimp_337K_MAF001_chimp/pannot/baseline_0_1000'
#         '/%s_0_1000_common_common.chr%d.pannot_mat.npz'%(AN,CHR)
#     )
#     temp_maf_G = mat_G_pos + mat_G_neg
#     print('mat_G=%d, mat_G_pos=%d, mat_G_neg=%d, mat_G_ref=%d' % (
#         mat_G.sum(), mat_G_pos.sum(), mat_G_neg.sum(), mat_G_ref.sum()
#     ))
#     print('mat_G - mat_G_ref n=%d' %(mat_G - mat_G.multiply(mat_G_ref)).sum())
#     print('mat_G_ref - mat_G n=%d' %(mat_G_ref - mat_G.multiply(mat_G_ref)).sum())
#     print('mat_G - mat_G_pos+mat_G_neg n=%d' %(mat_G - mat_G.multiply(temp_maf_G)).sum())
#     print('mat_G_pos+mat_G_neg - mat_G n=%d' %(temp_maf_G - mat_G.multiply(temp_maf_G)).sum())
#     print('')
#     break

baseline_0_1000
mat_G=1228, mat_G_pos=528, mat_G_neg=700, mat_G_ref=1228
mat_G - mat_G_ref n=0
mat_G_ref - mat_G n=0
mat_G - mat_G_pos+mat_G_neg n=0
mat_G_pos+mat_G_neg - mat_G n=0



In [6]:
# Check correctness
CHR = 22

# PGEN
PGEN_FILE = '/n/scratch/users/j/jz286/imp_geno_chimp/ukb_imp_chr@_v3_chimp'
df_snp_chr = ldspec.util.read_pgen(PGEN_FILE.replace("@", "%d" % CHR))["pvar"]
df_snp_chr["MAF"] = ldspec.util.read_pgen(PGEN_FILE.replace("@", "%d" % CHR))["afreq"]["MAF"]
df_snp_chr["mbin"] = [ldspec.util.get_mbin(x) for x in df_snp_chr["MAF"]]
df_snp_chr.index = df_snp_chr['SNP']

df_annot = ldspec.util.read_annot(
    '/n/groups/price/martin/LDSPEC_data/UKBimp_337K_MAF001/baseline_annot/baseline_165annots_chr%d.annot.gz' % CHR
)

# LD
mat_ld, dic_range = ldspec.util.read_ld(
    '/n/groups/price/UKBiobank/insample_ld/ld_files/ukb_imp_v3_chimp.c%d_s0_e10000_ld.npz' % CHR
)

In [10]:
DIC_PANNOT[pAN]

'/n/groups/price/martin/LDSPEC_data/UKBimp_337K_MAF001_chimp/pannot_fct/baseline_0_1000_ld_high_pos/@a.proxy_0_1000.ld_p50_p100.maf_lf_block.c@.pannot_mat.npz'

In [8]:
# for AN in AN_list[:1]:
# for AN in ['SuperEnhancer_Hnisz_common', 'H3K27ac_Hnisz']:
for AN in ['SuperEnhancer_Hnisz']:
    temp_v = df_annot[[x for x in df_annot if AN in x]].sum(axis=1)
    for pAN in DIC_PANNOT:
        print(AN, pAN)
        print(DIC_PANNOT[pAN])
        fpath = DIC_PANNOT[pAN].replace('@a', AN).replace('@', '%d' % CHR)
        mat_G = ldspec.util.read_pannot_mat(fpath)
        print('    mat_G', mat_G.shape, mat_G.getformat())

        n_print = 0
        i=0
        while i < 20000:      
            if mat_G.indptr[i]==mat_G.indptr[i+1]:
                i += 1
                continue    

            j = mat_G.indices[mat_G.indptr[i]:mat_G.indptr[i+1]][0] 
            
            print('    ------------------------------------------------------------------------------')
            print('    MAF = %0.4f, %0.4f,    dist = %d,    LD=%0.4f' % (
                df_snp_chr['MAF'][i], df_snp_chr['MAF'][j], df_snp_chr['BP'][i] - df_snp_chr['BP'][j], 
                mat_ld[i,j] if i<10000 else np.nan
            ))

#             print('    MAF = %0.4f, %0.4f,    dist = %d' % (
#                 df_snp_chr['MAF'][i], df_snp_chr['MAF'][j], df_snp_chr['BP'][i] - df_snp_chr['BP'][j], 
#             ))
            print('    %s: ' % AN, temp_v[i], temp_v[j])
            
            i+=10
            n_print+=1
            
            if n_print>2:
                break

SuperEnhancer_Hnisz baseline_0_100_ld_high_pos_common
/n/groups/price/martin/LDSPEC_data/UKBimp_337K_MAF001_chimp/pannot_fct/baseline_0_100_ld_high_pos/@a.proxy_0_100.ld_p50_p100.maf_common_block.c@.pannot_mat.npz
    mat_G (199680, 199680) csr
    ------------------------------------------------------------------------------
    MAF = 0.3761, 0.3761,    dist = -49,    LD=0.9668
    SuperEnhancer_Hnisz:  1 1
    ------------------------------------------------------------------------------
    MAF = 0.2256, 0.3711,    dist = -70,    LD=0.6602
    SuperEnhancer_Hnisz:  1 1
    ------------------------------------------------------------------------------
    MAF = 0.1961, 0.1961,    dist = -42,    LD=0.9977
    SuperEnhancer_Hnisz:  1 1
SuperEnhancer_Hnisz baseline_0_100_ld_high_pos_lf
/n/groups/price/martin/LDSPEC_data/UKBimp_337K_MAF001_chimp/pannot_fct/baseline_0_100_ld_high_pos/@a.proxy_0_100.ld_p50_p100.maf_lf_block.c@.pannot_mat.npz
    mat_G (199680, 199680) csr
    -------------